In [3]:
pip install nltk


     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.5 MB 3.3 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.5 MB 2.1 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.5 MB 2.0 MB/s eta 0:00:01
   ---------- ----------------------------- 0.4/1.5 MB 2.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.5 MB 2.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.6/1.5 MB 2.4 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.4 MB/s eta 0:00:01
   ---------------------- ----------------- 0.9/1.5 MB 2.4 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.5 MB 2.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 2.3 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
file_path = 'C:/Users/lavan/Downloads/Spam Email Detection - spam.csv'
data = pd.read_csv(file_path)
data
print(data.head())
print(data.info())


     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
m

In [11]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

data['processed_text'] = data['v2'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lavan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
data['processed_text']

0       go jurong point crazy available bugis n great ...
1                                 ok lar joking wif u oni
2       free entry 2 wkly comp win fa cup final tkts 2...
3                     u dun say early hor u c already say
4             nah dont think goes usf lives around though
                              ...                        
5567    2nd time tried 2 contact u u �750 pound prize ...
5568                          � b going esplanade fr home
5569                          pity mood soany suggestions
5570    guy bitching acted like id interested buying s...
5571                                       rofl true name
Name: processed_text, Length: 5572, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Transform the processed text into TF-IDF features
X = vectorizer.fit_transform(data['processed_text'])
y = data['v1']


In [15]:
X
y 

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)


MultinomialNB()

In [19]:
precision = precision_score(y_test, y_pred, pos_label='spam')


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='spam')
recall = recall_score(y_test, y_pred, pos_label='spam')
f1 = f1_score(y_test, y_pred, pos_label='spam')

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


Accuracy: 0.97
Precision: 1.00
Recall: 0.75
F1 Score: 0.85


In [23]:
def predict_spam(text):
    processed_text = preprocess_text(text)
    text_features = vectorizer.transform([processed_text])
    prediction = model.predict(text_features)
    return 'Spam' if prediction[0] else 'Not Spam'


new_email = "Congratulations! You've won a $1,000 gift card. Click here to claim your prize."
print(predict_spam(new_email))


Spam
